In [1]:
# Cell 1: Setup and Imports - Using testing_utils
import sys
import time
import numpy as np
import warnings
from IPython.display import clear_output

clear_output(wait=True)
warnings.filterwarnings('ignore')

# Import testing utilities - provides all shared infrastructure
from qectostim.testing import (
    # Status indicators
    STATUS_OK, STATUS_WARN, STATUS_SKIP, STATUS_FAIL,
    # Module management
    clear_qectostim_modules,
    # Test function
    test_decoder_on_code,
    # Decoder/code loading utilities
    load_all_decoders,
    discover_and_categorize_codes,
    print_code_summary,
    # Results formatting
    format_status,
    compute_summary_stats,
)

# Clear module cache for fresh testing
clear_qectostim_modules()

# Discover and categorize all codes WITH FAILURE REPORTING
# NOTE: Bosonic (GKP, rotor), qudit (Galois), and fracton (XCube, Haah) codes
# are excluded by default as they don't work with standard qubit decoders
categories, all_codes, failures = discover_and_categorize_codes(
    max_qubits=100,
    include_qldpc=True,
    include_subsystem=True,
    include_floquet=True,
    include_bosonic=False,   # Exclude - continuous variables
    include_qudit=False,     # Exclude - d>2 dimensions
    include_fracton=False,   # Exclude - exotic excitations
    report_failures=True,    # Report codes that failed to instantiate
)

# Print summary
print_code_summary(categories, "DECODER SMOKE TEST - ALL CODE TYPES")

# Show failed codes if any
if failures:
    print("\n" + "=" * 70)
    print("FAILED TO INSTANTIATE")
    print("=" * 70)
    print(f"\n{len(failures)} codes failed to load:\n")
    
    # Group by error type for readability
    skipped = {k: v for k, v in failures.items() if v.startswith("SKIPPED:")}
    timeout = {k: v for k, v in failures.items() if v.startswith("TIMEOUT:")}
    not_found = {k: v for k, v in failures.items() if v.startswith("NOT_FOUND:")}
    errors = {k: v for k, v in failures.items() 
              if not v.startswith(("SKIPPED:", "TIMEOUT:", "NOT_FOUND:"))}
    
    if errors:
        print(f"ERRORS ({len(errors)} codes - bugs to fix):")
        for name, error in sorted(errors.items()):
            print(f"  ❌ {name}: {error}")
    
    if not_found:
        print(f"\nNOT FOUND ({len(not_found)} codes):")
        for name in sorted(not_found.keys()):
            print(f"  ❓ {name}")
    
    if timeout:
        print(f"\nTIMEOUT ({len(timeout)} codes):")
        for name in sorted(timeout.keys()):
            print(f"  ⏱️ {name}")
    
    if skipped:
        print(f"\nSKIPPED ({len(skipped)} codes - known slow/special):")
        for name in sorted(skipped.keys()):
            print(f"  ⏭️ {name}")

DECODER SMOKE TEST - ALL CODE TYPES

Total discovered: 79 codes
  CSS: 50
  Color: 4
  Non-CSS: 5
  Subsystem: 4
  Floquet: 6
  QLDPC: 10

CSS Codes:
Code Name                                   n   k   d
-------------------------------------------------------
  FourQubit422_[[4,2,2]]                    4   2   2
  C6                                        6   2   2
  Steane_713                                7   1   3
  Shor_91                                   9   1   3
  ReedMuller_15_1_3                        15   1   3
  Hamming_CSS_7                             7   1   3
  Hamming_CSS_15                           15   7   3
  Hamming_CSS_31                           31  21   3
  Code_832                                  8   3   2
  Repetition_3                              3   1   3
  Repetition_5                              5   1   5
  Repetition_7                              7   1   7
  RotatedSurface_[[9,1,3]]                  9   1   3
  RotatedSurface_[[25,1,5]]           

In [2]:
# Debug cell: Check QLDPC factory loading
from qectostim.codes import qldpc as qldpc_module

print("Checking QLDPC factories in qldpc module:")
qldpc_factories = ['HGPHamming7', 'BBGrossCode', 'HDX_4', 'HDX_6', 'QuantumTanner_4', 'DLV_8',
                   'ExpanderLP_10_3', 'DHLV_5_1', 'BalancedProductRep5']

for name in qldpc_factories:
    factory = getattr(qldpc_module, name, None)
    if factory is None:
        print(f"  ❌ {name}: NOT FOUND in qldpc module")
    else:
        try:
            code = factory()
            print(f"  ✅ {name}: Success - {type(code).__name__} with n={getattr(code, 'n', '?')}")
        except Exception as e:
            print(f"  ⚠️ {name}: Found but failed - {e}")

# Use the actual patterns from discovery.py
print("\nChecking if QLDPC patterns would match:")
qldpc_names = ['Hypergraph', 'Bicycle', 'Lifted', 'Fiber', 'HGP', 'BB', 'GB',
               'HDX', 'Expander', 'DHLV', 'Campbell', 'Tanner', 'DLV', 
               'Lossless', 'HigherDim', 'Balanced', 'QLDPC']
for name in ['HDXCode', 'ExpanderLPCode', 'DHLVCode', 'QuantumTannerCode', 'DinurLinVidickCode']:
    matches = any(pat in name for pat in qldpc_names)
    print(f"  {name}: {'✅' if matches else '❌'} matches QLDPC pattern")

Checking QLDPC factories in qldpc module:
  ✅ HGPHamming7: Success - TillichZemorHGP with n=58
  ✅ BBGrossCode: Success - BivariateBicycleCode with n=144
  ✅ HDX_4: Success - HDXCode with n=48
  ✅ HDX_6: Success - HDXCode with n=108
  ✅ QuantumTanner_4: Success - QuantumTannerCode with n=32
  ✅ DLV_8: Success - DinurLinVidickCode with n=58
  ✅ ExpanderLP_10_3: Success - ExpanderLPCode with n=4525
  ✅ DHLV_5_1: Success - DHLVCode with n=285
  ✅ BalancedProductRep5: Success - BalancedProductCode with n=41

Checking if QLDPC patterns would match:
  HDXCode: ✅ matches QLDPC pattern
  ExpanderLPCode: ✅ matches QLDPC pattern
  DHLVCode: ✅ matches QLDPC pattern
  QuantumTannerCode: ✅ matches QLDPC pattern
  DinurLinVidickCode: ❌ matches QLDPC pattern
  ✅ ExpanderLP_10_3: Success - ExpanderLPCode with n=4525
  ✅ DHLV_5_1: Success - DHLVCode with n=285
  ✅ BalancedProductRep5: Success - BalancedProductCode with n=41

Checking if QLDPC patterns would match:
  HDXCode: ✅ matches QLDPC pattern
  E

In [3]:
# Cell 2: Load Available Decoders - Using testing_utils

decoder_classes = load_all_decoders()
print(f"Loaded {len(decoder_classes)} decoders: {list(decoder_classes.keys())}")

Loaded 11 decoders: ['PyMatching', 'FusionBlossom', 'BeliefMatching', 'BPOSD', 'Tesseract', 'UnionFind', 'MLE', 'Hypergraph', 'Chromobius', 'Concatenated', 'FlatConcat']


In [22]:
# Cell 3: Test Helper Function - Now imported from testing_utils
# The test_decoder_on_code function is already imported from qectostim.testing
# It handles:
#   - Color codes using ColorCodeMemoryExperiment for Chromobius-compatible DEMs
#   - CSS codes using CSSMemoryExperiment
#   - Other codes using StabilizerMemoryExperiment
#   - Chromobius will SKIP on non-color codes (expected behavior)

print("✓ test_decoder_on_code imported from qectostim.testing")
print("  - Color codes use ColorCodeMemoryExperiment (Chromobius-compatible DEMs)")
print("  - Other codes use CSSMemoryExperiment or StabilizerMemoryExperiment")
print("  - Chromobius will SKIP on non-color codes (expected)")

✓ test_decoder_on_code imported from qectostim.testing
  - Color codes use ColorCodeMemoryExperiment (Chromobius-compatible DEMs)
  - Other codes use CSSMemoryExperiment or StabilizerMemoryExperiment
  - Chromobius will SKIP on non-color codes (expected)


In [ ]:
# Cell 4: DECODER × CODE COMPATIBILITY MATRIX - ALL CODES
from IPython.display import clear_output
clear_output(wait=True)

p = 0.01
shots = 1000

# Store results - reinitialize to be safe
full_results = {}

# Build output lines to print at end (prevents output duplication)
output_lines = []
output_lines.append("="*130)
output_lines.append("DECODER × CODE COMPATIBILITY MATRIX - ALL CODE TYPES")
output_lines.append("="*130)
output_lines.append(f"\nTesting {len(all_codes)} decoder/code combinations at p={p}, {shots} shots...")
output_lines.append("Code types: CSS, Non-CSS, Subsystem, Floquet, QLDPC, Color")
output_lines.append("Note: Chromobius requires color-code-like DEMs (shows SKIP for other codes)\n")

# Header
dec_names = list(decoder_classes.keys())
header = f"{'Code':<35} {'Type':<10}"
for dec_name in dec_names:
    header += f" | {dec_name[:10]:^12}"
output_lines.append(header)
output_lines.append("-" * len(header))

# Process each code ONCE
for code_name, (code_type, code) in all_codes.items():
    row = f"{code_name:<35} {code_type:<10}"
    full_results[code_name] = {'type': code_type}
    
    for dec_name in dec_names:
        # test_decoder_on_code returns TestResult object
        result = test_decoder_on_code(
            code=code,
            decoder_class=decoder_classes[dec_name],
            decoder_name=dec_name,
            code_type=code_type,
            p=p,
            shots=shots
        )
        # Convert to dict for storage
        result_dict = result.to_dict()
        full_results[code_name][dec_name] = result_dict
        
        if result.status == 'OK':
            ler = result.ler
            if result.warnings:
                cell = f"⚠️{ler:.4f}" if ler is not None else "⚠️N/A"
            else:
                cell = f"✓ {ler:.4f}" if ler is not None else "✓ N/A"    
        elif result.status == 'WARN':
            cell = f"⚠️WARN"
        elif result.status == 'SKIP':
            cell = f"- SKIP"
        else:
            cell = f"✗ FAIL"
        
        row += f" | {cell:^12}"
    
    output_lines.append(row)

output_lines.append("-" * len(header))
output_lines.append(f"\nLegend: ✓=pass, ⚠️=warning, -=skip (expected), ✗=fail")
output_lines.append(f"Total codes tested: {len(full_results)}")

# Single print statement
print("\n".join(output_lines))

In [31]:
# Quick validation of CNOT direction fix
# Test codes that had higher LER with FusionBlossom in the new table
import sys
# Clear all cached qectostim modules
modules_to_clear = [k for k in list(sys.modules.keys()) if 'qectostim' in k]
for mod in modules_to_clear:
    del sys.modules[mod]

from qectostim.experiments import CSSMemoryExperiment
from qectostim.codes.discovery import get_code_classes
from qectostim.noise.models import CircuitDepolarizingNoise
import numpy as np
import sinter

code_classes = get_code_classes()

print("="*70)
print("VALIDATION: Logical Operator + CNOT Direction Fix (using sinter)")
print("="*70)
print("\nComparing LER values - should be close to old table values:")
print("  RotatedSurface_d3 @ p=0.01: OLD=0.012")
print("  ToricCode_3x3 @ p=0.01: OLD=0.004")
print()

# Test parameters
p = 0.01
shots = 10000

test_cases = [
    ('RotatedSurfaceCode', {'distance': 3}, 'Z'),
    ('ToricCode', {'Lx': 3, 'Ly': 3}, 'Z'),
]

for code_name, params, basis in test_cases:
    if code_name not in code_classes:
        print(f"⚠️ {code_name} not found")
        continue
    
    try:
        code = code_classes[code_name](**params)
        print(f"\n{code.name} ({code.n} data qubits):")
    except Exception as e:
        print(f"✗ Failed to instantiate {code_name}: {e}")
        continue
    
    try:
        exp = CSSMemoryExperiment(code, rounds=3, basis=basis)
        # FIXED: Use p2 for two-qubit gate noise (main error source)
        noise = CircuitDepolarizingNoise(p1=p/10, p2=p)
        circuit = exp.to_stim()
        noisy = noise.apply(circuit)
        dem = noisy.detector_error_model(decompose_errors=True)
        
        print(f"  Circuit: {circuit.num_qubits} qubits, DEM: {dem.num_detectors} detectors")
        
        # Use sinter for proper decoding with error handling
        task = sinter.Task(
            circuit=noisy,
            detector_error_model=dem,
            decoder='pymatching',
        )
        
        stats = sinter.collect(
            tasks=[task],
            num_workers=1,
            max_shots=shots,
            max_errors=shots,
            decoders=['pymatching'],
        )[0]
        
        ler = stats.errors / stats.shots if stats.shots > 0 else 0
        print(f"  ✓ LER: {ler:.4f} ({stats.errors}/{stats.shots} errors)")
        
    except Exception as e:
        import traceback
        print(f"  ✗ FAILED: {e}")
        traceback.print_exc()

print("\n" + "="*70)
print("If LER values are close to OLD table, fix is working!")
print("="*70)

VALIDATION: Logical Operator + CNOT Direction Fix (using sinter)

Comparing LER values - should be close to old table values:
  RotatedSurface_d3 @ p=0.01: OLD=0.012
  ToricCode_3x3 @ p=0.01: OLD=0.004


RotatedSurfaceCode (9 data qubits):
  Circuit: 17 qubits, DEM: 24 detectors
  ✓ LER: 0.0512 (512/10000 errors)

ToricCode (18 data qubits):
  Circuit: 34 qubits, DEM: 48 detectors
  ✓ LER: 0.0512 (512/10000 errors)

ToricCode (18 data qubits):
  Circuit: 34 qubits, DEM: 48 detectors
  ✓ LER: 0.0835 (835/10000 errors)

If LER values are close to OLD table, fix is working!
  ✓ LER: 0.0835 (835/10000 errors)

If LER values are close to OLD table, fix is working!


In [38]:
# Compare DEM structures between NEW and LEGACY
import sys
modules_to_clear = [k for k in list(sys.modules.keys()) if 'qectostim' in k]
for mod in modules_to_clear:
    del sys.modules[mod]

from qectostim.codes.surface import RotatedSurfaceCode
from qectostim.experiments import CSSMemoryExperiment
from qectostim.noise.models import CircuitDepolarizingNoise

code = RotatedSurfaceCode(distance=3)
exp = CSSMemoryExperiment(code, rounds=3, basis="Z")

circuit_new = exp.to_stim()
circuit_legacy = exp.to_stim_legacy()

noise = CircuitDepolarizingNoise(p1=0.001, p2=0.01)
noisy_new = noise.apply(circuit_new)
noisy_legacy = noise.apply(circuit_legacy)

dem_new = noisy_new.detector_error_model(decompose_errors=True)
dem_legacy = noisy_legacy.detector_error_model(decompose_errors=True)

print("="*70)
print("DEM STRUCTURE COMPARISON")
print("="*70)

print(f"\nNEW DEM: {dem_new.num_detectors} detectors, {dem_new.num_observables} observables")
print(f"LEGACY DEM: {dem_legacy.num_detectors} detectors, {dem_legacy.num_observables} observables")

# Count error mechanisms
def count_error_mechanisms(dem):
    single_det = 0
    two_det = 0
    with_obs = 0
    total = 0
    for instr in dem.flattened():
        if instr.type == "error":
            total += 1
            targets = list(instr.targets_copy())
            det_count = sum(1 for t in targets if t.is_relative_detector_id())
            has_obs = any(t.is_logical_observable_id() for t in targets)
            
            if det_count == 1:
                single_det += 1
            elif det_count == 2:
                two_det += 1
            
            if has_obs:
                with_obs += 1
                
    return {"total": total, "single": single_det, "two_det": two_det, "with_obs": with_obs}

new_counts = count_error_mechanisms(dem_new)
legacy_counts = count_error_mechanisms(dem_legacy)

print(f"\nError mechanisms:")
print(f"  NEW: {new_counts}")
print(f"  LEGACY: {legacy_counts}")

# The key metric: errors that can flip the observable without detection
# (single-detector errors with L0)
def count_dangerous_errors(dem):
    dangerous = []
    for instr in dem.flattened():
        if instr.type == "error":
            targets = list(instr.targets_copy())
            det_count = sum(1 for t in targets if t.is_relative_detector_id())
            has_obs = any(t.is_logical_observable_id() for t in targets)
            
            # Errors that flip observable with 0 or 1 detector are hard to decode
            if has_obs and det_count <= 1:
                prob = instr.args_copy()[0]
                dangerous.append((det_count, prob, str(instr)))
    return dangerous

new_dangerous = count_dangerous_errors(dem_new)
legacy_dangerous = count_dangerous_errors(dem_legacy)

print(f"\nDangerous errors (observable flip with ≤1 detector):")
print(f"  NEW: {len(new_dangerous)} errors")
print(f"  LEGACY: {len(legacy_dangerous)} errors")

if new_dangerous:
    print("\n  NEW dangerous:")
    for d, p, s in new_dangerous[:5]:
        print(f"    [{d} det, p={p:.6f}] {s[:80]}...")
        
if legacy_dangerous:
    print("\n  LEGACY dangerous:")
    for d, p, s in legacy_dangerous[:5]:
        print(f"    [{d} det, p={p:.6f}] {s[:80]}...")

DEM STRUCTURE COMPARISON

NEW DEM: 24 detectors, 1 observables
LEGACY DEM: 24 detectors, 1 observables

Error mechanisms:
  NEW: {'total': 100, 'single': 14, 'two_det': 34, 'with_obs': 26}
  LEGACY: {'total': 116, 'single': 28, 'two_det': 68, 'with_obs': 30}

Dangerous errors (observable flip with ≤1 detector):
  NEW: 2 errors
  LEGACY: 8 errors

  NEW dangerous:
    [1 det, p=0.015830] error(0.0158299) D20 L0...
    [1 det, p=0.015830] error(0.0158299) D21 L0...

  LEGACY dangerous:
    [1 det, p=0.002674] error(0.00267382) D0 L0...
    [1 det, p=0.005333] error(0.00533333) D1 L0...
    [1 det, p=0.007979] error(0.00797863) D8 L0...
    [1 det, p=0.007979] error(0.00797863) D9 L0...
    [1 det, p=0.007979] error(0.00797863) D16 L0...


In [33]:
# Cell 5: LER COMPARISON TABLE - ALL CODES
from IPython.display import clear_output
clear_output(wait=True)

print("="*140)
print("LER COMPARISON TABLE (p=0.01) - ALL CODE TYPES")
print("="*140)
print(f"\nTotal codes in all_codes: {len(all_codes)}")
print(f"Total codes in full_results: {len(full_results)}")
print("\nLower is better. Best decoder for each code highlighted.")
print("SKIP = decoder incompatible (e.g., Chromobius requires color-code DEMs)\n")

# Header
dec_names = list(decoder_classes.keys())
header = f"{'Code':<35} | {'Type':<10} | {'d':>2} | {'No-decode':>10}"
for dec_name in dec_names:
    header += f" | {dec_name[:10]:>10}"
header += " | Best"
print(header)
print("-" * len(header))

# Use full_results directly since it has all the data from Cell 4
for code_name in sorted(full_results.keys()):
    code_results = full_results[code_name]
    code_type = code_results.get('type', 'Unknown')
    
    # Get code object for distance
    code = all_codes.get(code_name, (None, None))[1]
    d = code.metadata.get('distance', '?') if code else '?'
    
    # Get no-decode LER
    ler_no_decode = None
    for key, res in code_results.items():
        if isinstance(res, dict) and res.get('ler_no_decode') is not None:
            ler_no_decode = res['ler_no_decode']
            break
    
    nd_str = f"{ler_no_decode:.4f}" if ler_no_decode else 'N/A'
    row = f"{code_name:<35} | {code_type:<10} | {str(d):>2} | {nd_str:>10}"
    
    # Find best decoder
    best_ler = float('inf')
    best_decoder = None
    
    for dec_name in dec_names:
        res = code_results.get(dec_name, {})
        status = res.get('status') if isinstance(res, dict) else None
        ler = res.get('ler') if isinstance(res, dict) else None
        
        if status == 'SKIP':
            ler_str = 'SKIP'
        elif ler is not None:
            ler_str = f"{ler:.4f}"
            if ler < best_ler:
                best_ler = ler
                best_decoder = dec_name
        else:
            ler_str = 'FAIL'
        row += f" | {ler_str:>10}"
    
    best_str = best_decoder[:10] if best_decoder else 'N/A'
    row += f" | {best_str}"
    print(row)

print("-" * len(header))
print(f"\nTotal rows: {len(full_results)}")

LER COMPARISON TABLE (p=0.01) - ALL CODE TYPES

Total codes in all_codes: 79
Total codes in full_results: 79

Lower is better. Best decoder for each code highlighted.
SKIP = decoder incompatible (e.g., Chromobius requires color-code DEMs)

Code                                | Type       |  d |  No-decode | PyMatching | FusionBlos | BeliefMatc |      BPOSD |  Tesseract |  UnionFind |        MLE | Hypergraph | Chromobius | Concatenat | Best
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
BB_6x6                              | QLDPC      |  ? |     0.0780 |       FAIL |     0.0870 |       FAIL |     0.0150 |     0.0130 |       FAIL |       SKIP |       FAIL |       SKIP |       SKIP | Tesseract
BaconShor_3x3                       | Subsystem  |  3 |     0.1010 |     0.0270 |     0.0480 |     0.0230 |     0.0130 |     0.0140 |     0.024

In [34]:
# Quick summary of errors
print("=" * 60)
print("QUICK ERROR SUMMARY")
print("=" * 60)
print(f"Errors: {len(errors)}")
print(f"Failures: {len(failures)}")
print(f"Timeouts: {len(timeout)}")
print(f"Skipped: {len(skipped)}")
print(f"Not found: {len(not_found)}")
print()

if errors:
    print("ERROR codes:")
    for name, msg in errors.items():
        print(f"  ❌ {name}: {msg[:80]}...")

QUICK ERROR SUMMARY
Errors: 0
Failures: 13
Timeouts: 3
Skipped: 10
Not found: 0



In [35]:
# Check the FAILURES - what codes are failing and why?
print("=" * 60)
print("FAILURE DETAILS")
print("=" * 60)

for code_name, info in failures.items():
    print(f"\n{code_name}:")
    print(f"  {info[:200]}..." if len(info) > 200 else f"  {info}")


FAILURE DETAILS

HypergraphProductCode:
  SKIPPED: Known to be very slow or require special handling

ExpanderLP_10_3:
  TIMEOUT: Timed out after 5.0s

ExpanderLP_15_4:
  TIMEOUT: Timed out after 5.0s

DHLV_7_2:
  TIMEOUT: Timed out after 5.0s

HigherDimHom_3D:
  SKIPPED: Known to be very slow or require special handling

HigherDimHom_4D:
  SKIPPED: Known to be very slow or require special handling

ModularQudit3DSurfaceCode:
  SKIPPED: Known to be very slow or require special handling

ModularSurface3D_L3_d3:
  SKIPPED: Known to be very slow or require special handling

ModularSurface3D_L4_d5:
  SKIPPED: Known to be very slow or require special handling

HomologicalNumberPhaseCode:
  SKIPPED: Known to be very slow or require special handling

NumberPhase_L3_T2:
  SKIPPED: Known to be very slow or require special handling

NumberPhase_L4_T3:
  SKIPPED: Known to be very slow or require special handling

HomologicalProductCode:
  SKIPPED: Known to be very slow or require special handling

In [36]:
# Look at actual failures (not skips or timeouts)
print("=" * 60)
print("ACTUAL DECODER FAILURES (High LER)")  
print("=" * 60)

actual_failures = {}
for code_name, msg in failures.items():
    if not msg.startswith("SKIPPED") and not msg.startswith("TIMEOUT"):
        actual_failures[code_name] = msg
        
if actual_failures:
    for code_name, msg in actual_failures.items():
        print(f"\n{code_name}:")
        print(f"  {msg[:200]}..." if len(msg) > 200 else f"  {msg}")
else:
    print("\n✓ NO ACTUAL FAILURES!")
    print("All failures are due to SKIPS (special handling needed) or TIMEOUTS (slow instantiation).")

print(f"\nActual failures: {len(actual_failures)}")
print(f"Skips + Timeouts: {len(failures) - len(actual_failures)}")


ACTUAL DECODER FAILURES (High LER)

✓ NO ACTUAL FAILURES!
All failures are due to SKIPS (special handling needed) or TIMEOUTS (slow instantiation).

Actual failures: 0
Skips + Timeouts: 13


In [37]:
# Final summary using the correct data sources
print("=" * 60)
print("🎉 FINAL SMOKE TEST SUMMARY")
print("=" * 60)

# Parse failure categories
skipped = {k: v for k, v in failures.items() if v.startswith("SKIPPED:")}
timeout = {k: v for k, v in failures.items() if v.startswith("TIMEOUT:")}
errors_only = {k: v for k, v in failures.items() 
               if not v.startswith(("SKIPPED:", "TIMEOUT:", "NOT_FOUND:"))}
not_found = {k: v for k, v in failures.items() if v.startswith("NOT_FOUND:")}

total_loaded = len(all_codes)
total_failures = len(failures)
n_errors = len(errors_only)
n_skips = len(skipped)
n_timeouts = len(timeout)
n_not_found = len(not_found)

# Working codes = total loaded (i.e. codes that successfully instantiated)
# These are what actually got tested
print(f"\n{'Metric':<30} {'Count':>6}")
print("-" * 38)
print(f"{'✅ Codes loaded successfully':<30} {total_loaded:>6}")
print(f"{'❌ Errors (bugs)':<30} {n_errors:>6}")
print(f"{'⏱️ Timeouts (slow)':<30} {n_timeouts:>6}")
print(f"{'⏭️ Skipped (special handling)':<30} {n_skips:>6}")
print(f"{'❓ Not found':<30} {n_not_found:>6}")
print("-" * 38)

if n_errors == 0:
    print(f"\n🎉 ALL CONSTRUCTOR/INSTANTIATION ERRORS FIXED!")
    print(f"   {total_loaded} codes loaded and available for testing")
    print(f"   {n_skips + n_timeouts} codes skipped (known slow or need special handling)")
else:
    print(f"\n⚠️  {n_errors} codes have errors to fix:")
    for name, err in errors_only.items():
        print(f"   • {name}: {err[:80]}...")


🎉 FINAL SMOKE TEST SUMMARY

Metric                          Count
--------------------------------------
✅ Codes loaded successfully        79
❌ Errors (bugs)                     0
⏱️ Timeouts (slow)                  3
⏭️ Skipped (special handling)      10
❓ Not found                         0
--------------------------------------

🎉 ALL CONSTRUCTOR/INSTANTIATION ERRORS FIXED!
   79 codes loaded and available for testing
   13 codes skipped (known slow or need special handling)


In [38]:
# Check for 4D code failures specifically
print("4D code related failures:")
for name, msg in failures.items():
    if '4D' in name or 'Toric4' in name or 'LoopToric' in name:
        print(f"  {name}: {msg[:100]}...")

4D code related failures:
  HigherDimHom_4D: SKIPPED: Known to be very slow or require special handling...


In [39]:
# Cell 6: DECODER SPEED COMPARISON
from IPython.display import clear_output
clear_output(wait=True)

print("="*110)
print("DECODER SPEED COMPARISON (time in ms for 1000 shots)")
print("="*110)
print(f"\nTotal codes in full_results: {len(full_results)}\n")

# Header
dec_names = list(decoder_classes.keys())
header = f"{'Code':<30} {'Type':<10}"
for dec_name in dec_names:
    header += f" | {dec_name[:10]:>10}"
print(header)
print("-" * len(header))

# Use full_results directly since it has all the data from Cell 4
for code_name in sorted(full_results.keys()):
    code_results = full_results[code_name]
    code_type = code_results.get('type', 'Unknown')
    row = f"{code_name:<30} {code_type:<10}"
    
    for dec_name in dec_names:
        res = code_results.get(dec_name, {})
        time_ms = res.get('time_ms') if isinstance(res, dict) else None
        time_str = f"{time_ms:.1f}" if time_ms else 'FAIL'
        row += f" | {time_str:>10}"
    
    print(row)

print("-" * len(header))
print(f"\nTotal rows: {len(full_results)}")

DECODER SPEED COMPARISON (time in ms for 1000 shots)

Total codes in full_results: 79

Code                           Type       | PyMatching | FusionBlos | BeliefMatc |      BPOSD |  Tesseract |  UnionFind |        MLE | Hypergraph | Chromobius | Concatenat
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------
BB_6x6                         QLDPC      |       FAIL |      697.3 |       FAIL |    14747.0 |     6943.4 |       FAIL |       FAIL |       FAIL |       FAIL |       FAIL
BaconShor_3x3                  Subsystem  |        0.5 |        4.6 |       16.6 |       22.0 |       18.3 |        0.2 |        0.4 |        1.2 |       FAIL |       FAIL
BalancedProduct_5x5_G1         QLDPC      |        2.0 |       73.1 |      332.0 |     1013.8 |       58.7 |        1.9 |       FAIL |        3.1 |       FAIL |       FAIL
BalancedProduct_7x7_G1         QLDPC      |        9.

In [40]:
# Cell 7: NOISE LEVEL SCALING TEST

print("="*110)
print("NOISE LEVEL SCALING TEST")
print("="*110)

# Pick a representative CSS code (RotatedSurface d=3) 
test_code_name = None
test_code_type = None
for name, (ctype, code) in all_codes.items():
    if 'RotatedSurface' in name and code.metadata.get('distance') == 3:
        test_code_name = name
        test_code_type = ctype
        break

if test_code_name is None:
    test_code_name = list(all_codes.keys())[0]
    test_code_type = all_codes[test_code_name][0]

test_code = all_codes[test_code_name][1]
print(f"\nTesting {test_code_name} ({test_code_type}) across noise levels with all decoders...\n")

noise_levels = [0.001, 0.005, 0.01, 0.02]

# Header
dec_names = list(decoder_classes.keys())
header = f"{'p':<10}"
for dec_name in dec_names:
    header += f" | {dec_name[:12]:>12}"
header += " | No-decode"
print(header)
print("-" * len(header))

for p in noise_levels:
    row = f"{p:<10.4f}"
    ler_no_decode = None
    
    for dec_name in dec_names:
        # test_decoder_on_code returns TestResult dataclass, not dict
        result = test_decoder_on_code(
            code=test_code,
            code_type=test_code_type,
            decoder_name=dec_name,
            decoder_class=decoder_classes[dec_name],
            p=p,
            shots=2000,
            rounds=3
        )
        
        # Access attributes directly (TestResult is a dataclass)
        if ler_no_decode is None and result.ler_no_decode is not None:
            ler_no_decode = result.ler_no_decode
        
        ler = result.ler
        ler_str = f"{ler:.6f}" if ler is not None else 'FAIL'
        row += f" | {ler_str:>12}"
    
    nd_str = f"{ler_no_decode:.6f}" if ler_no_decode else 'N/A'
    row += f" | {nd_str}"
    print(row)

print("-" * len(header))

NOISE LEVEL SCALING TEST

Testing RotatedSurface_[[9,1,3]] (CSS) across noise levels with all decoders...

p          |   PyMatching | FusionBlosso | BeliefMatchi |        BPOSD |    Tesseract |    UnionFind |          MLE |   Hypergraph |   Chromobius | Concatenated | No-decode
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------
0.0010     |     0.000000 |     0.006500 |     0.000000 |     0.000000 |     0.000000 |     0.000000 |     0.000000 |     0.011000 |         FAIL |         FAIL | 0.009500
0.0010     |     0.000000 |     0.006500 |     0.000000 |     0.000000 |     0.000000 |     0.000000 |     0.000000 |     0.011000 |         FAIL |         FAIL | 0.009500
0.0050     |     0.006000 |     0.030000 |     0.005000 |     0.003500 |     0.003000 |     0.005000 |     0.006000 |     0.053000 |         FAIL |         FAIL | 0.044500
0.0050     |     0.006000 |    

In [41]:
# Cell 8: FINAL SUMMARY - ALL CODE TYPES

print("="*100)
print("FINAL DECODER SMOKE TEST SUMMARY - ALL CODE TYPES")
print("="*100)

# Count results by status
passed = 0
with_warnings = 0
skipped = 0
failed = 0
total_tests = 0

# Track by code type - include Color codes
results_by_type = {'CSS': {'pass': 0, 'warn': 0, 'skip': 0, 'fail': 0},
                   'Color': {'pass': 0, 'warn': 0, 'skip': 0, 'fail': 0},
                   'Non-CSS': {'pass': 0, 'warn': 0, 'skip': 0, 'fail': 0},
                   'Subsystem': {'pass': 0, 'warn': 0, 'skip': 0, 'fail': 0},
                   'Floquet': {'pass': 0, 'warn': 0, 'skip': 0, 'fail': 0},
                   'QLDPC': {'pass': 0, 'warn': 0, 'skip': 0, 'fail': 0}}

failed_tests = []

for code_name, code_results in full_results.items():
    code_type = code_results.get('type', 'Unknown')
    if code_type not in results_by_type:
        results_by_type[code_type] = {'pass': 0, 'warn': 0, 'skip': 0, 'fail': 0}
    
    for dec_name in decoder_classes.keys():
        res = code_results.get(dec_name, {})
        if isinstance(res, dict):
            total_tests += 1
            status = res.get('status')
            if status == 'OK':
                if res.get('warnings'):
                    with_warnings += 1
                    results_by_type[code_type]['warn'] += 1
                else:
                    passed += 1
                    results_by_type[code_type]['pass'] += 1
            elif status == 'WARN':
                with_warnings += 1
                results_by_type[code_type]['warn'] += 1
            elif status == 'SKIP':
                skipped += 1
                results_by_type[code_type]['skip'] += 1
            else:
                failed += 1
                results_by_type[code_type]['fail'] += 1
                failed_tests.append((code_name, dec_name, res.get('error', 'unknown')))

print(f"\n📊 OVERALL RESULTS:")
print(f"   Total decoder × code tests: {total_tests}")
print(f"   ✓ Passed:   {passed} ({100*passed/total_tests:.1f}%)")
print(f"   ⚠ Warnings: {with_warnings} ({100*with_warnings/total_tests:.1f}%)")
print(f"   - Skipped:  {skipped} ({100*skipped/total_tests:.1f}%) [expected - decoder incompatible]")
print(f"   ✗ Failed:   {failed} ({100*failed/total_tests:.1f}%)")

print(f"\n📊 RESULTS BY CODE TYPE:")
for ctype, counts in results_by_type.items():
    total = counts['pass'] + counts['warn'] + counts['skip'] + counts['fail']
    if total > 0:
        chromobius_note = " (Chromobius works!)" if ctype == 'Color' else ""
        print(f"   {ctype:>10}: {counts['pass']} pass, {counts['warn']} warn, {counts['skip']} skip, {counts['fail']} fail (total {total}){chromobius_note}")

print(f"\n🔧 DECODERS TESTED: {len(decoder_classes)}")
for dec_name in decoder_classes.keys():
    note = " (works on Color codes now!)" if dec_name == "Chromobius" else ""
    print(f"   - {dec_name}{note}")

print(f"\n📦 CODES TESTED BY TYPE:")
# Use categories from testing_utils (already loaded)
for code_type, codes_dict in categories.items():
    if codes_dict:
        print(f"\n   {code_type} ({len(codes_dict)} codes):")
        for name, code in list(codes_dict.items())[:10]:  # Limit to first 10
            d = code.metadata.get('distance', '?') if hasattr(code, 'metadata') else '?'
            print(f"     - {name}: [[{code.n},{code.k},{d}]]")
        if len(codes_dict) > 10:
            print(f"     ... and {len(codes_dict) - 10} more")

print("\n" + "="*100)
if failed == 0:
    print("✓ ALL TESTS PASSED (no unexpected failures)")
    if skipped > 0:
        print(f"  ({skipped} tests skipped - Chromobius skipped on non-color codes)")
else:
    print(f"⚠ {failed} TESTS FAILED (unexpected):")
    for code_name, dec_name, error in failed_tests[:20]:
        print(f"   - {code_name} × {dec_name}: {error}")
    if len(failed_tests) > 20:
        print(f"   ... and {len(failed_tests) - 20} more")
print("="*100)

FINAL DECODER SMOKE TEST SUMMARY - ALL CODE TYPES

📊 OVERALL RESULTS:
   Total decoder × code tests: 790
   ✓ Passed:   427 (54.1%)
   ⚠ Warnings: 116 (14.7%)
   - Skipped:  231 (29.2%) [expected - decoder incompatible]
   ✗ Failed:   16 (2.0%)

📊 RESULTS BY CODE TYPE:
          CSS: 303 pass, 51 warn, 132 skip, 14 fail (total 500)
        Color: 19 pass, 14 warn, 6 skip, 1 fail (total 40) (Chromobius works!)
      Non-CSS: 0 pass, 10 warn, 40 skip, 0 fail (total 50)
    Subsystem: 22 pass, 9 warn, 9 skip, 0 fail (total 40)
      Floquet: 25 pass, 21 warn, 14 skip, 0 fail (total 60)
        QLDPC: 58 pass, 11 warn, 30 skip, 1 fail (total 100)

🔧 DECODERS TESTED: 10
   - PyMatching
   - FusionBlossom
   - BeliefMatching
   - BPOSD
   - Tesseract
   - UnionFind
   - MLE
   - Hypergraph
   - Chromobius (works on Color codes now!)
   - Concatenated

📦 CODES TESTED BY TYPE:

   CSS (50 codes):
     - FourQubit422_[[4,2,2]]: [[4,2,2]]
     - C6: [[6,2,2]]
     - Steane_713: [[7,1,3]]
     - 

In [42]:
# Cell 9: CHROMOBIUS DIRECT TEST ON COLOR CODES
# This cell tests Chromobius directly on color codes using ColorCodeMemoryExperiment
# which generates 4D detector coordinates with proper color annotations

import sys
# Force reimport to pick up latest code changes
modules_to_remove = [k for k in sys.modules.keys() if 'qectostim' in k]
for mod in modules_to_remove:
    del sys.modules[mod]

from qectostim.codes.color.triangular_colour import TriangularColourCode
from qectostim.codes.color.hexagonal_colour import HexagonalColourCode
from qectostim.codes.color.colour_code import ColourCode488
from qectostim.experiments.memory import ColorCodeMemoryExperiment
from qectostim.noise.models import CircuitDepolarizingNoise
import chromobius
import numpy as np

print("="*60)
print("CHROMOBIUS DIRECT TEST ON COLOR CODES")
print("="*60)
print("Uses ColorCodeMemoryExperiment for 4D detector coordinates")
print()

codes = [
    ('TriangularColour_d3', TriangularColourCode(distance=3)),
    ('HexagonalColour_d2', HexagonalColourCode(distance=2)),
    ('HexagonalColour_d3', HexagonalColourCode(distance=3)),
    ('Colour488_d3', ColourCode488(distance=3)),
]

shots = 1000
results = []

for name, code in codes:
    print(f"\n{'-'*50}")
    print(f"Testing: {name} (n={code.n}, k={code.k})")
    meta = code.metadata
    print(f"  stab_colors: {meta.get('stab_colors')}")
    print(f"  is_chromobius_compatible: {meta.get('is_chromobius_compatible')}")
    
    if not meta.get('is_chromobius_compatible', False):
        print(f"  ⚠️  SKIPPED - not 3-colorable (stabilizers form K4 complete graph)")
        results.append((name, 'SKIP', 'Not 3-colorable'))
        continue
    
    try:
        noise = CircuitDepolarizingNoise(p1=0.01, p2=0.01)
        exp = ColorCodeMemoryExperiment(code=code, rounds=3, noise_model=noise)
        circuit = noise.apply(exp.to_stim())
        
        # Sample from circuit directly
        sampler = circuit.compile_detector_sampler()
        dets, obs = sampler.sample(shots, separate_observables=True)
        
        # Compile Chromobius decoder
        dem = circuit.detector_error_model(decompose_errors=True)
        decoder = chromobius.compile_decoder_for_dem(dem)
        
        # Decode
        dets_packed = np.packbits(dets.astype(np.uint8), axis=1, bitorder='little')
        predictions = decoder.predict_obs_flips_from_dets_bit_packed(dets_packed)
        
        # Calculate error rate
        obs = obs.astype(np.uint8)
        errors = np.any(predictions != obs, axis=1).sum()
        ler = errors / shots
        print(f"  ✓ Chromobius decode: SUCCESS - LER = {ler:.4f} ({errors}/{shots} errors)")
        results.append((name, 'PASS', f'{ler:.4f}'))
        
    except Exception as e:
        print(f"  ✗ ERROR: {type(e).__name__}: {e}")
        results.append((name, 'FAIL', str(e)[:50]))

print("\n" + "="*60)
print("SUMMARY")
print("="*60)
for name, status, detail in results:
    emoji = {'PASS': '✓', 'SKIP': '⚠️', 'FAIL': '✗'}[status]
    print(f"  {emoji} {name}: {status} - {detail}")

CHROMOBIUS DIRECT TEST ON COLOR CODES
Uses ColorCodeMemoryExperiment for 4D detector coordinates


--------------------------------------------------
Testing: TriangularColour_d3 (n=7, k=1)
  stab_colors: [0, 1, 2]
  is_chromobius_compatible: True
  ✓ Chromobius decode: SUCCESS - LER = 0.0190 (19/1000 errors)

--------------------------------------------------
Testing: HexagonalColour_d2 (n=8, k=2)
  stab_colors: [0, 0, 1]
  is_chromobius_compatible: True
  ✓ Chromobius decode: SUCCESS - LER = 0.0190 (19/1000 errors)

--------------------------------------------------
Testing: HexagonalColour_d2 (n=8, k=2)
  stab_colors: [0, 0, 1]
  is_chromobius_compatible: True
  ✓ Chromobius decode: SUCCESS - LER = 0.0000 (0/1000 errors)

--------------------------------------------------
Testing: HexagonalColour_d3 (n=17, k=7)
  stab_colors: [0, 0, 0, 1, 1]
  is_chromobius_compatible: True
  ✓ Chromobius decode: SUCCESS - LER = 0.0000 (0/1000 errors)

-----------------------------------------------

In [43]:
# Debug: Try to instantiate the fracton codes directly
import sys
for mod in list(sys.modules.keys()):
    if 'qectostim' in mod:
        del sys.modules[mod]

print("="*70)
print("TRYING TO INSTANTIATE FRACTON CODES DIRECTLY")
print("="*70)

# Import topological module
from qectostim.codes import topological

# Try each fracton code
fracton_classes = ['HaahCode', 'XCubeCode', 'ChamonCode', 'CheckerboardCode', 
                    'FibonacciFractalCode', 'SierpinskiPrismCode']

for class_name in fracton_classes:
    cls = getattr(topological, class_name, None)
    if cls is None:
        print(f"❌ {class_name}: NOT FOUND")
        continue
    
    # Try different parameter combinations
    params_to_try = [
        {'L': 3},
        {'L': 4},
        {'L': 3, 'T': 2},
        {},
    ]
    
    for params in params_to_try:
        try:
            code = cls(**params) if params else cls()
            print(f"✓ {class_name}({params}): n={code.n}, k={code.k}")
            break
        except Exception as e:
            print(f"  {class_name}({params}): {type(e).__name__}: {str(e)[:50]}")

# Also try HoneycombCode
print("\n" + "="*70)
print("TRYING FLOQUET CODES")
print("="*70)

from qectostim.codes import floquet

for class_name in ['HoneycombCode', 'ISGFloquetCode']:
    cls = getattr(floquet, class_name, None)
    if cls is None:
        print(f"❌ {class_name}: NOT FOUND")
        continue
    
    params_to_try = [
        {'Lx': 2, 'Ly': 3},
        {'distance': 3},
        {'L': 3},
        {},
    ]
    
    for params in params_to_try:
        try:
            code = cls(**params) if params else cls()
            print(f"✓ {class_name}({params}): n={code.n}, k={code.k}")
            break
        except Exception as e:
            print(f"  {class_name}({params}): {type(e).__name__}: {str(e)[:60]}")

TRYING TO INSTANTIATE FRACTON CODES DIRECTLY
✓ HaahCode({'L': 3}): n=54, k=4
✓ XCubeCode({'L': 3}): n=81, k=15
✓ ChamonCode({'L': 3}): n=54, k=6
  CheckerboardCode({'L': 3}): ValueError: L must be even and >= 2
✓ CheckerboardCode({'L': 4}): n=64, k=10
  FibonacciFractalCode({'L': 3}): TypeError: FibonacciFractalCode.__init__() got an unexpected 
  FibonacciFractalCode({'L': 4}): TypeError: FibonacciFractalCode.__init__() got an unexpected 
  FibonacciFractalCode({'L': 3, 'T': 2}): TypeError: FibonacciFractalCode.__init__() got an unexpected 
✓ FibonacciFractalCode({}): n=23, k=3
  SierpinskiPrismCode({'L': 3}): TypeError: SierpinskiPrismCode.__init__() got an unexpected k
  SierpinskiPrismCode({'L': 4}): TypeError: SierpinskiPrismCode.__init__() got an unexpected k
  SierpinskiPrismCode({'L': 3, 'T': 2}): TypeError: SierpinskiPrismCode.__init__() got an unexpected k
✓ SierpinskiPrismCode({}): n=80, k=2

TRYING FLOQUET CODES
  HoneycombCode({'Lx': 2, 'Ly': 3}): TypeError: HoneycombCode.

In [44]:
# Cell: Verify fixes for k<=0 and d=? codes
import sys
# Force reimport
for mod in list(sys.modules.keys()):
    if 'qectostim' in mod:
        del sys.modules[mod]

from qectostim.codes import discover_all_codes

print("="*70)
print("VERIFYING k AND d FIXES")
print("="*70)

codes = discover_all_codes(max_qubits=100, timeout_per_code=3.0)

print(f"\nTotal codes discovered: {len(codes)}")

# Check for problematic codes
k_zero_codes = []
k_negative_codes = []
d_unknown_codes = []
all_good_codes = []

for name, code in codes.items():
    k = code.k
    d = code.metadata.get('distance', None)
    
    if k < 0:
        k_negative_codes.append((name, k, d))
    elif k == 0:
        k_zero_codes.append((name, k, d))
    elif d is None or d == '?':
        d_unknown_codes.append((name, k, d))
    else:
        all_good_codes.append((name, k, d))

print(f"\n✅ Codes with k>0 and known d: {len(all_good_codes)}")
print(f"⚠️  Codes with k=0: {len(k_zero_codes)}")
print(f"❌ Codes with k<0: {len(k_negative_codes)}")
print(f"⚠️  Codes with unknown d: {len(d_unknown_codes)}")

if k_negative_codes:
    print(f"\n❌ NEGATIVE K CODES (still broken):")
    for name, k, d in k_negative_codes:
        print(f"   {name}: k={k}, d={d}")

if k_zero_codes:
    print(f"\n⚠️  ZERO K CODES:")
    for name, k, d in k_zero_codes:
        print(f"   {name}: k={k}, d={d}")

if d_unknown_codes:
    print(f"\n⚠️  UNKNOWN DISTANCE CODES:")
    for name, k, d in d_unknown_codes[:10]:
        print(f"   {name}: k={k}, d={d}")
    if len(d_unknown_codes) > 10:
        print(f"   ... and {len(d_unknown_codes) - 10} more")

print("\n" + "="*70)
if len(k_negative_codes) == 0:
    print("✅ NO NEGATIVE K CODES - XCubeCode fix verified!")
else:
    print("❌ Still have negative k codes - needs investigation")

if len(k_zero_codes) <= 2:  # Some codes legitimately have k=0
    print("✅ ZERO K CODES MINIMIZED")
else:
    print(f"⚠️  Still have {len(k_zero_codes)} k=0 codes")

print("="*70)

VERIFYING k AND d FIXES

VERIFYING k AND d FIXES



Total codes discovered: 71

✅ Codes with k>0 and known d: 52
⚠️  Codes with k=0: 7
❌ Codes with k<0: 0
⚠️  Codes with unknown d: 12

⚠️  ZERO K CODES:
   ColorCode3D_d3: k=0, d=3
   TetrahedralColorCode: k=0, d=None
   BallColor_[[12,0,None]]: k=0, d=None
   BallColor_[[24,0,None]]: k=0, d=None
   BallColor_3D_[[12,0,None]]: k=0, d=None
   BallColor_4D_[[24,0,None]]: k=0, d=None
   BB_6x6: k=0, d=None

⚠️  UNKNOWN DISTANCE CODES:
   LoopToricCode4D: k=2, d=None
   GoldenCode_5: k=1, d=None
   TwistedToric_[[32,2,None]]: k=2, d=None
   TwistedToric_4x4_[[32,2,None]]: k=2, d=None
   ProjectivePlaneSurface_[[13,1,None]]: k=1, d=None
   ProjectivePlane_4_[[25,1,None]]: k=1, d=None
   HolographicRainbowCode_L4_d2: k=1, d=None
   HolographicRainbowCode_L6_d3: k=1, d=None
   HGPHamming7_[[58,16,None]]: k=16, d=None
   HGPRep5_[[41,1,None]]: k=1, d=None
   ... and 2 more

✅ NO NEGATIVE K CODES - XCubeCode fix verified!
⚠️  Still have 7 k=0 codes
